In [1]:
import torch   
print(torch.__version__)

2.10.0+cu128


In [2]:
# Vérifie si CUDA est disponible
print(torch.cuda.is_available()) # Retourne True si CUDA est activé


# Affiche la version de CUDA utilisée par PyTorch
print(torch.version.cuda)

True
12.8


# 1. IMPORT DEPENDENCIES

In [2]:
import os
import json
from PIL import Image
import torchvision
from torchvision import transforms as T
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO

In [3]:
import os 

# DEBUG PATHS\n
print("Current Working Directory:", os.getcwd())
print("Files in CWD:", os.listdir(os.getcwd()))

dataset_path = 'Hold Detector.v2i.coco-segmentation'
if os.path.exists(dataset_path):
    print(f"Dataset folder '{dataset_path}' FOUND.")
else:
    print(f"Dataset folder '{dataset_path}' NOT FOUND in CWD.")
    # Attempt to look in common parent folders if not found
    possible_paths = ['MaskRCNN/Hold Detector.v2i.coco-segmentation', '../MaskRCNN/Hold Detector.v2i.coco-segmentation']
    for p in possible_paths:
        if os.path.exists(p):
            print(f"Found at alternative path: {p}")
            dataset_path = p
            break

train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')
print(f"Using Train Path: {train_path}")

Current Working Directory: c:\Users\silue\Documents\telecom_clustering\segmentation\MaskRCNN
Files in CWD: ['climb.ipynb', 'climbing_model.pth', 'climb_epoch1.ipynb', 'epoch1.py', 'Hold Detector.v2i.coco-segmentation', 'perf.txt', 'requirements.txt']
Dataset folder 'Hold Detector.v2i.coco-segmentation' FOUND.
Using Train Path: Hold Detector.v2i.coco-segmentation\train


# 2. DEFINE DATASET CLASS
We implement a custom Dataset that reads COCO annotations and converts polygons to masks.

In [4]:
class ClimbingDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        
        # Initialize COCO api
        ann_file = os.path.join(root, "_annotations.coco.json")
        if not os.path.exists(ann_file):
            raise FileNotFoundError(f"Annotation file not found at: {ann_file}")

        self.coco = COCO(ann_file)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Load Image
        coco = self.coco
        img_id = self.ids[index]
        img_metadata = coco.loadImgs(img_id)[0]
        path = img_metadata['file_name']
        img_path = os.path.join(self.root, path)
        
        if not os.path.exists(img_path):
             # Fallback if image is not right in root (COCO sometimes has flat structure)
             print(f"Warning: Image not found at {img_path}")

        img = Image.open(img_path).convert("RGB")

        # Load Annotations
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        num_objs = len(anns)
        boxes = []
        masks = []
        labels = []

        for ann in anns:
            xmin, ymin, w, h = ann['bbox']
            boxes.append([xmin, ymin, xmin + w, ymin + h])
            labels.append(ann['category_id'])
            masks.append(coco.annToMask(ann))

        # Convert to Tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # Masks need to be uint8 for proper tensor conversion later
        if num_objs > 0:
            masks = np.array(masks)
            masks = torch.as_tensor(masks, dtype=torch.uint8)
        else:
            # Handle empty image case (optional, usually training sets have objs)
            masks = torch.zeros((0, img_metadata['height'], img_metadata['width']), dtype=torch.uint8)
            boxes = torch.zeros((0, 4), dtype=torch.float32)

        image_id = torch.tensor([img_id])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)
    
def get_transform(train):
    custom_transforms = []
    custom_transforms.append(T.ToTensor())
    # We can add augmentation here if needed
    return T.Compose(custom_transforms)

def collate_fn(batch):
    return tuple(zip(*batch))

In [5]:
import os
import json

def check_dataset(split='train'):
    base_dir = r"C:\Users\silue\Documents\telecom_clustering\segmentation\MaskRCNN"
    dataset_dir = os.path.join(base_dir, "Hold Detector.v2i.coco-segmentation", split)
    ann_file = os.path.join(dataset_dir, "_annotations.coco.json")
    
    if not os.path.exists(ann_file):
        print(f"Annotation file not found: {ann_file}")
        return

    with open(ann_file, 'r') as f:
        data = json.load(f)
    
    total_images = len(data['images'])
    missing_count = 0
    valid_count = 0
    
    print(f"Checking {split} set...")
    print(f"Total entries in JSON: {total_images}")
    
    for img in data['images']:
        fname = img['file_name']
        img_path = os.path.join(dataset_dir, fname)
        if not os.path.exists(img_path):
            missing_count += 1
            # print(f"Missing: {fname}")
        else:
            valid_count += 1
            
    print(f"Valid images: {valid_count}")
    print(f"Missing images: {missing_count}")

check_dataset('train')
check_dataset('test')

Checking train set...
Total entries in JSON: 117
Valid images: 114
Missing images: 3
Checking test set...
Total entries in JSON: 1
Valid images: 1
Missing images: 0


# 3. CONFIGURE MODEL
We fine-tune a pre-trained Mask R-CNN model.

In [6]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

    return model

# 4. TRAINING LOOP

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 2 classes: background + hold
num_classes = 2

# Use our robust paths logic from above variable
print(f"Initializing datasets with train path: {train_path} and test path: {test_path}")

# Use our dataset and defined transformations
dataset_train = ClimbingDataset(train_path, get_transform(train=True))
dataset_test = ClimbingDataset(test_path, get_transform(train=False))

# Define training and validation data loaders
data_loader = DataLoader(
    dataset_train, batch_size=2, shuffle=True,  # Low batch size for Mask RCNN usually safe
    collate_fn=collate_fn
)

data_loader_test = DataLoader(
    dataset_test, batch_size=1, shuffle=False,
    collate_fn=collate_fn
)

# Get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# Move model to the right device
model.to(device)

# Construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training function
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    model.train()
    for i, (images, targets) in enumerate(data_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % print_freq == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Loss: {losses.item()}")

# Train for 1 Epoch (for demonstration)
num_epochs = 1
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()

print("Training Complete!")

# Save Model
torch.save(model.state_dict(), "climbing_model.pth")

Initializing datasets with train path: Hold Detector.v2i.coco-segmentation\train and test path: Hold Detector.v2i.coco-segmentation\test
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


c:\Users\silue\Documents\telecom_clustering\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\silue\Documents\telecom_clustering\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'Hold Detector.v2i.coco-segmentation\\train\\climbing-hold-climbing-bouldering-sport-climbing-adventure-rock-climbing-climbing-shoe-free-climbing-recreation-rock-climbing-equipment-wall-individual-sports-competition-leisure-competition-event-plastic-concrete-rock-back-play-1621676_jpg.rf.cc9a2c229ee40a15e1e73a56c909b2c6.jpg'

# 5. VISUALIZATION
Visualize predictions on a test image.

In [ ]:
model.eval()
img, _ = dataset_test[0]

with torch.no_grad():
    prediction = model([img.to(device)])

print("Number of detected holds:", len(prediction[0]['masks']))

# Visualize
img_np = img.mul(255).permute(1, 2, 0).byte().numpy()
plt.figure(figsize=(12, 12))
plt.imshow(img_np)

# Overlay masks (simple logic)
for i in range(len(prediction[0]['masks'])):
    mask = prediction[0]['masks'][i, 0].mul(255).byte().cpu().numpy()
    score = prediction[0]['scores'][i].item()
    if score > 0.5: # Filter low confidence
        plt.imshow(mask, cmap='jet', alpha=0.3)
plt.show()

NameError: name 'model' is not defined